In [3]:
import sklearn
sklearn.__version__

'0.23.2'

In [128]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
import warnings
warnings.filterwarnings('ignore')

data_path = 'C:/Users/Francis/OneDrive/Desktop/Code/ML馬拉松/Validation/modelpoint/'

df = pd.read_csv(data_path + 'house_train.csv')

In [129]:
df.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


In [130]:
Y_train = df['SalePrice']
Id = df['Id']
df = df.drop(['SalePrice','Id'],axis = 1)

In [131]:
df.head()

,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,...,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition
0,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,Inside,...,0,0,NaN,NaN,NaN,0,2,2008,WD,Normal
1,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,FR2,...,0,0,NaN,NaN,NaN,0,5,2007,WD,Normal
2,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,Inside,...,0,0,NaN,NaN,NaN,0,9,2008,WD,Normal
3,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,Corner,...,0,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml
4,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,FR2,...,0,0,NaN,NaN,NaN,0,12,2008,WD,Normal


In [132]:
df['SaleCondition'] = df['SaleCondition'].fillna('None')
mean_df = df.groupby(['SaleCondition'])['GrLivArea'].mean().reset_index()
mode_df = df.groupby(['SaleCondition'])['GrLivArea'].apply(lambda x: x.mode()[0]).reset_index()
median_df = df.groupby(['SaleCondition'])['GrLivArea'].median().reset_index()
max_df = df.groupby(['SaleCondition'])['GrLivArea'].max().reset_index()
temp = pd.merge(mean_df,mode_df,how = 'left', on = ['SaleCondition'])
temp = pd.merge(temp,median_df , how = 'left',on = ['SaleCondition'])
temp = pd.merge(temp,max_df, how = 'left' , on = ['SaleCondition'])
temp.columns = ['SaleCondition','Area_Sale_Mean','Area_Sale_Mode','Area_Sale_Median','Area_Sale_Max']
temp

,SaleCondition,Area_Sale_Mean,Area_Sale_Mode,Area_Sale_Median,Area_Sale_Max
0,Abnorml,1436.128713,864,1302.0,4476
1,AdjLand,1112.500000,980,1143.0,1184
2,Alloca,1701.750000,1535,1439.5,3194
3,Family,1480.950000,948,1390.5,2526
4,Normal,1492.968280,864,1456.0,4316
5,Partial,1795.696000,1456,1646.0,5642


In [133]:
df = pd.merge(df,temp,how = 'left',on = ['SaleCondition'])
df = df.drop(['SaleCondition'], axis =1)
df.head()

,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,...,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,Area_Sale_Mean,Area_Sale_Mode,Area_Sale_Median,Area_Sale_Max
0,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,Inside,...,NaN,NaN,0,2,2008,WD,1492.968280,864,1456.0,4316
1,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,FR2,...,NaN,NaN,0,5,2007,WD,1492.968280,864,1456.0,4316
2,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,Inside,...,NaN,NaN,0,9,2008,WD,1492.968280,864,1456.0,4316
3,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,Corner,...,NaN,NaN,0,2,2006,WD,1436.128713,864,1302.0,4476
4,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,FR2,...,NaN,NaN,0,12,2008,WD,1492.968280,864,1456.0,4316


In [134]:
num_feature = []

for dtype, feature in zip(df.dtypes,df.columns) : 
    if dtype == 'int64' or dtype == 'float64':
        num_feature.append(feature)

print(f'{len(num_feature)} Numeric Feature :{num_feature}')

df = df[num_feature]
df = df.fillna(-1)
MMEncoder = MinMaxScaler()
df

40 Numeric Feature :['MSSubClass', 'LotFrontage', 'LotArea', 'OverallQual', 'OverallCond', 'YearBuilt', 'YearRemodAdd', 'MasVnrArea', 'BsmtFinSF1', 'BsmtFinSF2', 'BsmtUnfSF', 'TotalBsmtSF', '1stFlrSF', '2ndFlrSF', 'LowQualFinSF', 'GrLivArea', 'BsmtFullBath', 'BsmtHalfBath', 'FullBath', 'HalfBath', 'BedroomAbvGr', 'KitchenAbvGr', 'TotRmsAbvGrd', 'Fireplaces', 'GarageYrBlt', 'GarageCars', 'GarageArea', 'WoodDeckSF', 'OpenPorchSF', 'EnclosedPorch', '3SsnPorch', 'ScreenPorch', 'PoolArea', 'MiscVal', 'MoSold', 'YrSold', 'Area_Sale_Mean', 'Area_Sale_Mode', 'Area_Sale_Median', 'Area_Sale_Max']


,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,BsmtFinSF2,...,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold,Area_Sale_Mean,Area_Sale_Mode,Area_Sale_Median,Area_Sale_Max
0,60,65.0,8450,7,5,2003,2003,196.0,706,0,...,0,0,0,0,2,2008,1492.968280,864,1456.0,4316
1,20,80.0,9600,6,8,1976,1976,0.0,978,0,...,0,0,0,0,5,2007,1492.968280,864,1456.0,4316
2,60,68.0,11250,7,5,2001,2002,162.0,486,0,...,0,0,0,0,9,2008,1492.968280,864,1456.0,4316
3,70,60.0,9550,7,5,1915,1970,0.0,216,0,...,0,0,0,0,2,2006,1436.128713,864,1302.0,4476
4,60,84.0,14260,8,5,2000,2000,350.0,655,0,...,0,0,0,0,12,2008,1492.968280,864,1456.0,4316
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1455,60,62.0,7917,6,5,1999,2000,0.0,0,0,...,0,0,0,0,8,2007,1492.968280,864,1456.0,4316
1456,20,85.0,13175,6,6,1978,1988,119.0,790,163,...,0,0,0,0,2,2010,1492.968280,864,1456.0,4316
1457,70,66.0,9042,7,9,1941,2006,0.0,275,0,...,0,0,0,2500,5,2010,1492.968280,864,1456.0,4316
1458,20,68.0,9717,5,6,1950,1996,0.0,49,1029,...,0,0,0,0,4,2010,1492.968280,864,1456.0,4316


In [135]:
# 沒有這四個新特徵的 dataframe 稱為 df_minus
df_minus = df.drop(['Area_Sale_Mean','Area_Sale_Mode','Area_Sale_Median','Area_Sale_Max'], axis = 1)

# 原始特徵 + 線性迴歸
train_X = MMEncoder.fit_transform(df_minus)
estimator = LinearRegression()
cross_val_score(estimator,train_X ,Y_train,cv = 5).mean()

0.7754139026838608

In [136]:
# 新特徵 + 線性迴歸 : 有些微改善
train_X = MMEncoder.fit_transform(df)
cross_val_score(estimator,train_X,Y_train,cv = 5).mean()

0.7944232820914647

In [137]:
# 原始特徵 + 隨機森林
train_X = MMEncoder.fit_transform(df_minus)
estimator2 = RandomForestRegressor()
cross_val_score(estimator2,train_X , Y_train , cv = 5).mean()

0.8514688534396659

In [138]:
# 新特徵 + 隨機森林
train_X = MMEncoder.fit_transform(df)
cross_val_score(estimator2,train_X,Y_train,cv = 5).mean()

0.8510402101212554

鐵達尼號生存預測

In [209]:
import pandas as pd
import numpy as np
import sklearn
from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestRegressor

data_path = 'C:/Users/Francis/OneDrive/Desktop/Code/ML馬拉松/Validation/modelpoint/'

In [210]:
df = pd.read_csv(data_path + "titanic_train.csv")
test_X = pd.read_csv(data_path + 'titanic_test.csv')

Id = df['PassengerId']
Y_train = df['Survived']
train_X = df.drop(['PassengerId','Survived'],axis = 1)

In [211]:
train_X

,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
...,...,...,...,...,...,...,...,...,...,...
886,2,"Montvila, Rev. Juozas",male,27.0,0,0,211536,13.0000,NaN,S
887,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.0000,B42,S
888,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,NaN,1,2,W./C. 6607,23.4500,NaN,S
889,1,"Behr, Mr. Karl Howell",male,26.0,0,0,111369,30.0000,C148,C


In [212]:
#以fare作為Embarked
mean_Em = train_X.groupby(['Embarked'])['Fare'].mean().reset_index()

In [213]:
mean_Em.columns = ['Embarked','Fare_Mean']

In [214]:
temp_sex = pd.get_dummies(train_X['Sex'])
train_X = pd.concat([train_X,temp_sex],axis = 1)
train_X = pd.merge(train_X,mean_Em,how = 'left',on = ['Embarked'])
train_X.dtypes

Pclass         int64
Name          object
Sex           object
Age          float64
SibSp          int64
Parch          int64
Ticket        object
Fare         float64
Cabin         object
Embarked      object
female         uint8
male           uint8
Fare_Mean    float64
dtype: object

In [215]:
num_feature = []

for dtype,feature in zip(train_X.dtypes,train_X.columns):
    if dtype == "int64" or dtype == "float64" or dtype == "uint8":
        num_feature.append(feature)

print(f'Numeric Feature {len(num_feature)} : {num_feature}')
train_X = train_X[num_feature]
train_X = train_X.fillna(-1)
MMEncoder = MinMaxScaler()

Numeric Feature 8 : ['Pclass', 'Age', 'SibSp', 'Parch', 'Fare', 'female', 'male', 'Fare_Mean']


In [216]:
#不加入新特徵以及不使用MinMaxScaler
estimator = LogisticRegression()
estimator2 = RandomForestRegressor()
train_X_m = train_X.drop(['Fare_Mean','female','male'], axis = 1)
train_X_m
print(cross_val_score(estimator,train_X_m,Y_train,cv = 5).mean())
print(cross_val_score(estimator2,train_X,Y_train,cv = 5).mean())

0.6981796497395015
0.3810300858006287


In [217]:
#不加入新特徵以及使用MinMaxScaler
train_X_m1 = MMEncoder.fit_transform(train_X_m)
print(cross_val_score(estimator,train_X_m1,Y_train,cv = 5).mean())
print(cross_val_score(estimator2,train_X_m1,Y_train,cv = 5).mean())

0.70041428661101
0.05763777961727097


In [219]:
#加入新特徵以及不使用MinMaxScaler
print(cross_val_score(estimator,train_X,Y_train,cv = 5).mean())
print(cross_val_score(estimator2,train_X,Y_train,cv = 5).mean())

0.7811374050593183
0.3891187703915592


In [220]:
#加入新特徵以及使用MinMaxScaler
train_X1 = MMEncoder.fit_transform(train_X)
print(cross_val_score(estimator,train_X1,Y_train,cv = 5).mean())
print(cross_val_score(estimator2,train_X1,Y_train,cv = 5).mean())

0.7744021090954742
0.38401936270431297


array([[1.        , 0.28395062, 0.125     , 0.        , 0.01415106],
       [0.        , 0.48148148, 0.125     , 0.        , 0.13913574],
       [1.        , 0.33333333, 0.        , 0.        , 0.01546857],
       ...,
       [1.        , 0.        , 0.125     , 0.33333333, 0.04577135],
       [0.        , 0.33333333, 0.        , 0.        , 0.0585561 ],
       [1.        , 0.40740741, 0.        , 0.        , 0.01512699]])